In [ ]:
    if __name__ == '__main__':
        %load_ext loader
    else:
        %load_ext rites.loader
    from rites.config import alias
        
    [alias.__setitem__(str, f"rites.{str}") for str in ('markdown', 'template', 'conventions', 'test')]

    if __name__ == '__main__':
        %reload_ext config
        !jupyter nbconvert --to python __init__.ipynb
        load_ipython_extension()
    else:
        %load_ext rites.config

The loader extension is already loaded. To reload it, use:
  %reload_ext loader
